In [1]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
from underthesea import word_tokenize

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
stopwords_path = "Data/Data_ref/vietnamese-stopwords.txt"

In [4]:
# df = pd.read_csv('Data/Data_dat_dai/data_after_preprocessing.csv', index_col=0)
# df.head()

In [5]:
# X_train, X_test, Y_train, Y_test= model_selection.train_test_split(df['text'],df['label'],test_size=0.2,random_state=1)

In [6]:
def read_splitted_data():

    df_train = pd.read_csv('Data/Data_dat_dai/df_train_after.csv')
    df_test = pd.read_csv('Data/Data_dat_dai/df_test_after.csv')


    X_train = df_train[['text']]
    Y_train = df_train[['label']]

    X_test = df_test[['text']]
    Y_test = df_test[['label']]   

    return X_train, Y_train, X_test, Y_test

In [7]:
def tf_idf_config(X_train, X_test):

    Tfidf_vect = TfidfVectorizer(max_features=1000)
    Tfidf_vect.fit(X_train['text'].to_list())
    tfidf_train = Tfidf_vect.transform(X_train['text'])
    tfidf_test = Tfidf_vect.transform(X_test['text'])

    tfidf_train = tfidf_train.toarray()
    tfidf_test = tfidf_test.toarray()

    return Tfidf_vect, tfidf_train, tfidf_test

In [8]:
def model_selection(model_name, tfidf_train, tfidf_test, Y_train, Y_test):
    if (model_name == "SVM"):
        model = svm.SVC(C=1.0, kernel='linear', gamma='auto', degree = 3)
        model.fit(tfidf_train,Y_train)
        # predict the labels on validation dataset
        predictions_SVM = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)  
        print(classification_report(Y_test,predictions_SVM)) 
        return model    

    elif (model_name == "LGBM"):

        model = LGBMClassifier()
        model.fit(tfidf_train,Y_train)
        predictions_lgbm = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, Y_test)*100)
        print(classification_report(Y_test,predictions_lgbm))

        return model

    elif (model_name == "Random Forest"):

        model = RandomForestClassifier(n_estimators=1000,min_samples_leaf=2,
                                        min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                                      random_state=25)
        model.fit(tfidf_train,Y_train) 
        predictions_rf = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("rf Accuracy Score -> ",accuracy_score(predictions_rf, Y_test)*100)
        print(classification_report(Y_test,predictions_rf))

        return model
    else:
        print("No invalid model selected, try again")
        return 0


In [9]:
X_train, Y_train, X_test, Y_test = read_splitted_data()
Tfidf_vect, tfidf_train, tfidf_test = tf_idf_config(X_train, X_test)
selected = model_selection(model_name = "LGBM", tfidf_train = tfidf_train, tfidf_test = tfidf_test, Y_train = Y_train, Y_test = Y_test)

C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ligbm Accuracy Score ->  87.71929824561403
              precision    recall  f1-score   support

          -1       0.85      0.85      0.85        46
           1       0.90      0.90      0.90        68

    accuracy                           0.88       114
   macro avg       0.87      0.87      0.87       114
weighted avg       0.88      0.88      0.88       114



### Test Prediction

In [10]:
def lower_df(df):
    df['text'] = df['text'].str.lower()
    return df

In [11]:
def segmentation(df):


    '''UNDER THE SEA'''
    
    list_text = df['text'].to_list()
    #print(list_text[0])
    for i in range(len(list_text)):
        list_text[i] = word_tokenize(list_text[i], format='text')
    
    df['text'] =  list_text
    return df

In [12]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [13]:
def remove_stopwords(df):
    stopwords = get_stopwords_list(stopwords_path)

    list_text = df['text'].tolist()

    results = []
    for text in list_text:
        tmp = text.split(' ')
        for stop_word in stopwords:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))

    df['text'] = results
    return df

In [14]:
def data_preprocessing(df):
    df = lower_df(df)
    df = segmentation(df)
    df = remove_stopwords(df)

    return df

In [15]:
def predict_sample(tmp_text):
    input = tmp_text
    list_input = [input]
    test_df = pd.DataFrame()
    test_df['text'] = list_input
    test_df = data_preprocessing(test_df)

    test_df = Tfidf_vect.transform(test_df['text'])
    test_df = test_df.toarray()
    prediction = selected.predict(test_df)   

    if (prediction[0] == -1):
        print("Bai viet co kha nang tieu cuc cao !!!")   
    else:
        print("Bai viet co kha nang tieu cuc thap :3")

In [16]:
input = "Mòn mỏi chờ đền bù Nhiều tháng qua, anh Võ Thành Tiến (xã Ia Lốp) liên tục gửi kiến nghị đến cơ quan chức năng yêu cầu phải chi trả tiền hỗ trợ, đền bù đất cho gia đình anh. Theo anh Tiến, năm 2022, cơ quan chức năng đã thu hồi khoảng 800m2 đất của gia đình anh để thực hiện dự án hồ chứa nước Ia Mơr (giai đoạn 2, hạng mục kênh và công trình trên kênh đoạn qua địa bàn tỉnh Đắk Lắk). Với diện tích đất bị thu hồi trên, gia đình anh Tiến được phê duyệt hỗ trợ, bồi thường 80 triệu đồng. Theo anh Tiến, từ năm 2020, khi bắt đầu triển khai dự án, chủ đầu tư đã vận động người dân bàn giao mặt bằng trước và được người dân đồng thuận cao. Dự án tuyến kênh đã thu hồi đất của hàng trăm hộ dân trên địa bàn. Trong đó, trung bình mỗi hộ dân bị thu hồi từ 500m2-1.000m2; số tiền đền bù dao động từ 50-100 triệu đồng. “Đến nay, dự án đã cơ bản hoàn thành, chủ đầu tư vẫn không đoái hoài đến việc chi trả tiền bồi thường cho người dân. Chúng tôi nhiều lần phản ánh thì chủ đầu tư hứa hẹn sẽ giải ngân, đền bù cho người dân trước tháng 12-2021. Nay đã quá nửa năm 2022, tiền đền bù vẫn chưa thấy đâu. Chúng tôi đa phần đều làm nông, cuộc sống khó khăn, nên rất cần tiền để xoay trở, trang trải cuộc sống”, anh Tiến bức xúc. Tương tự, ông Võ Văn Dân (xã Ia Lốp) cho biết, gia đình ông được phê duyệt đền bù 86 triệu đồng. Đất của gia đình ông cũng bị thu hồi từ năm 2020 nhưng công tác chi trả đền bù chậm. Theo ông, chính quyền địa phương và chủ đầu tư phải có trách nhiệm với sinh kế của hàng trăm hộ dân sau khi đất canh tác đã bị thu hồi. Tiền có nhưng… hãy chờ Theo tìm hiểu, dự án hồ thủy lợi Ia Mơr (giai đoạn 2, kênh và công trình trên kênh đoạn qua địa bàn tỉnh Đắk Lắk) do Ban Quản lý dự án đầu tư xây dựng công trình giao thông và NN-PTNT tỉnh Đắk Lắk (Ban QLDA tỉnh Đắk Lắk) làm chủ đầu tư, có tổng mức đầu tư 159 tỷ đồng, mục đích dẫn nước từ hồ thủy lợi Ia Mơr (tỉnh Gia Lai) cấp cho 4.000ha đất canh tác thuộc địa bàn xã Ia Lốp và một phần nhỏ đất canh tác tại xã Ia Jlơi (huyện Ea Súp). Theo ông Võ Thành Toàn, Trưởng phòng Đền bù - Giải phóng mặt bằng (Ban QLDA tỉnh Đắk Lắk), dự án trên tổng chiều dài hơn 33km, để thi công, đơn vị đã thu hồi, giải phóng mặt bằng (GPMB) đất của 354 hộ dân. Do tính cấp bách và để dự án thi công kịp tiến độ, đơn vị đã thu hồi đất trước rồi bồi thường sau. Năm 2020, Ban QLDA tỉnh Đắk Lắk đã lập phương án hỗ trợ bồi thường GPMB với kinh phí gần 20 tỷ đồng để trình UBND huyện Ea Súp thẩm định. Tuy nhiên, do gặp nhiều vướng mắc trong việc áp khung chính sách, nguồn gốc đất, xác định tuổi cây trồng… nên phương án bồi thường trên vẫn chưa được thẩm định. “Tiền chi trả bồi thường, GPMB cho các hộ dân ở huyện Ea Súp đã có sẵn nhưng phương án bồi thường chưa được UBND huyện Ea Súp thẩm định nên chúng tôi chưa giải ngân được”, ông Toàn nói. Lý giải việc chưa thẩm định phương án bồi thường dự án trên, ông Ngô Văn Thắng, Phó Chủ tịch UBND huyện Ea Súp, cho biết, diện tích đất thu hồi của người dân trong vùng dự án gặp vướng mắc trong việc quy hoạch sử dụng đất chưa được phê duyệt. Do đó, UBND huyện Ea Súp đã làm văn bản gửi UBND tỉnh Đắk Lắk để xin ý kiến xử lý. “Chúng tôi đã nộp quy hoạch sử dụng đất giai đoạn 2021-2030 cho UBND tỉnh để xem xét, phê duyệt. Huyện cũng cố gắng hoàn tất các thủ tục chi trả bồi thường cho người dân trong vùng dự án”, ông Thắng cho biết thêm. Dự án hồ thủy lợi Ia Mơr (huyện Chư Prông, tỉnh Gia Lai) có tổng mức đầu tư hơn 3.000 tỷ đồng, do Ban QLĐTXD thủy lợi 8 làm chủ đầu tư. Hồ Ia Mơr có sức chứa gần 180 triệu m3 nước, với mục tiêu cấp nước cho 10.000ha đất canh tác tại tỉnh Gia Lai và 4.000ha đất canh tác tại tỉnh Đắk Lắk. MAI CƯỜNG Từ khoá : Dự án Dự án thủy lợi Dự án thủy lợi Ia Mơr đền bù Các tin, bài viết khác Đưa 5 thuyền viên trên tàu Bình Thuận bị chìm về bờ từ tàu hàng nước ngoài Voọc Chà vá chân xám còn rất ít, nguy cơ bị săn bắn, bẫy rất cao Quảng Bình: Không đồng tình chuyển đất dịch vụ thương mại sang đất ở đô thị Đưa 5 thuyền viên trên tàu Bình Thuận bị chìm về bờ từ tàu hàng nước ngoài Khánh thành Nhà truyền thống Công an TPHCM Bến Tre: Trao 23 suất học bổng Nguyễn Thị Định Xe buýt chật vật tìm cách tồn tại Lập đoàn kiểm tra toàn tuyến vụ đường 500 tỷ chưa bàn giao đã nứt chằng chịt Khẩn trương xử lý vụ xây dựng trái phép tại danh thắng Ba Làng An Lập phương án đầu tư cao tốc Pleiku - Quy Nhơn Xem thêm"

predict_sample(input)


Bai viet co kha nang tieu cuc cao !!!
